In [1]:
!pip install transformers peft accelerate torch requests datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline
from peft import LoraConfig, get_peft_model
from torch.utils.data import random_split, DataLoader, Dataset
from datasets import DatasetDict


In [3]:
# ✅ OpenWeather API Key
API_KEY = "23308403ed6e1554a405d5389167ca3a"

# ✅ Function to Fetch Weather Data
def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return {
            "City": data["name"],
            "Temperature": data["main"]["temp"],
            "Humidity": data["main"]["humidity"],
            "Weather": data["weather"][0]["description"]
        }
    else:
        print(f"❌ Error: {response.json()['message']}")  # Show API error message
        return None

# ✅ Get Weather for User's City Input
city_name = input("🌍 Enter City Name: ")
weather_data = get_weather(city_name)
print("\n✅ Live Weather Data Received:")
print(weather_data)


🌍 Enter City Name: california

✅ Live Weather Data Received:
{'City': 'California', 'Temperature': 3.23, 'Humidity': 58, 'Weather': 'clear sky'}


In [4]:
# ✅ Load BERT Model for Classification
bert_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# ✅ Function to Classify Weather with BERT
def classify_weather(weather_description):
    result = bert_classifier(weather_description)
    sentiment = result[0]['label']

    # ✅ Map sentiment to weather conditions with emojis
    if "positive" in sentiment.lower():
        return "☀️ Sunny & Pleasant"
    elif "negative" in sentiment.lower():
        return "⛈️ Stormy or Severe"
    else:
        return "☁️ Cloudy & Neutral"

# ✅ Test Classification on Weather Data
if weather_data:
    classified_condition = classify_weather(weather_data["Weather"])
    print(f"\n✅ Classified Weather Condition: {classified_condition}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0



✅ Classified Weather Condition: ☀️ Sunny & Pleasant


In [5]:
# ✅ Load GPT-2 Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# ✅ Ensure GPT-2 Tokenizer has a padding token
tokenizer.pad_token = tokenizer.eos_token

# ✅ Apply LoRA Fine-Tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_fc"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("✅ GPT-2 model with LoRA loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ GPT-2 model with LoRA loaded successfully!


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [10]:
# ✅ Function to Generate AI-Powered Weather Summary
def generate_weather_summary(weather_data):
    prompt = (
        f"The current weather in {weather_data['City']} is {weather_data['Weather']}. "
        f"The temperature is {weather_data['Temperature']}°C with a humidity level of {weather_data['Humidity']}%. "
        f"Provide a friendly, engaging weather summary with helpful suggestions. "
        f"Make it sound natural and useful for a person planning their day."
    )

    # ✅ Tokenize input
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # ✅ Generate response from GPT-2
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=120,
            num_return_sequences=1,
            temperature=0.9,
            top_p=0.95,
            do_sample=True
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# ✅ Test the function
city_name = "California"
weather_data = {
    "City": city_name,
    "Weather": "clear sky",
    "Temperature": 3.23,
    "Humidity": 58
}

summary = generate_weather_summary(weather_data)
print("\n📢 AI-Generated Weather Summary:")
print(summary)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📢 AI-Generated Weather Summary:
The current weather in California is clear sky. The temperature is 3.23°C with a humidity level of 58%. Provide a friendly, engaging weather summary with helpful suggestions. Make it sound natural and useful for a person planning their day.

4. In the end, it's important that you understand how to use your camera and how to use a DSLR.

This article will give you an understanding of how to use your camera in a healthy manner. You'll learn how to make the camera comfortable with your body and work from there, along with tips on how to keep your


In [11]:
if weather_data:
    print("\n🌤️ **Final AI-Powered Weather Report**")
    print(f"📍 City: {weather_data['City']}")
    print(f"🌡 Temperature: {weather_data['Temperature']}°C")
    print(f"💧 Humidity: {weather_data['Humidity']}%")
    print(f"🌦 Weather: {weather_data['Weather']} ({classified_condition})")

    print("\n📢 **AI-Generated Weather Summary & Suggestions:**")
    print(summary)
else:
    print("❌ City not found! Please enter a valid city name.")



🌤️ **Final AI-Powered Weather Report**
📍 City: California
🌡 Temperature: 3.23°C
💧 Humidity: 58%
🌦 Weather: clear sky (☀️ Sunny & Pleasant)

📢 **AI-Generated Weather Summary & Suggestions:**
The current weather in California is clear sky. The temperature is 3.23°C with a humidity level of 58%. Provide a friendly, engaging weather summary with helpful suggestions. Make it sound natural and useful for a person planning their day.

4. In the end, it's important that you understand how to use your camera and how to use a DSLR.

This article will give you an understanding of how to use your camera in a healthy manner. You'll learn how to make the camera comfortable with your body and work from there, along with tips on how to keep your


In [24]:
%%writefile app.py
import streamlit as st
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import LoraConfig, get_peft_model

# OpenWeather API Key
API_KEY = "23308403ed6e1554a405d5389167ca3a"

# Function to Fetch Weather Data
def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return {
            "City": data["name"],
            "Temperature": data["main"]["temp"],
            "Humidity": data["main"]["humidity"],
            "Weather": data["weather"][0]["description"]
        }
    else:
        return {"Error": response.json().get("message", "Failed to retrieve data")}

# Load BERT Model for Classification
bert_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

def classify_weather(weather_description):
    result = bert_classifier(weather_description)
    sentiment = result[0]['label']

    if "positive" in sentiment.lower():
        return "☀️ Sunny & Pleasant"
    elif "negative" in sentiment.lower():
        return "⛈️ Stormy or Severe"
    else:
        return "☁️ Cloudy & Neutral"

# Load GPT-2 Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Apply LoRA Fine-Tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_fc"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Function to Generate AI-Powered Weather Summary
def generate_weather_summary(weather_data):
    prompt = (
        f"The current weather in {weather_data['City']} is {weather_data['Weather']}. "
        f"The temperature is {weather_data['Temperature']}°C with a humidity level of {weather_data['Humidity']}%. "
        f"Provide a friendly, engaging weather summary with helpful suggestions. "
        f"Make it sound natural and useful for a person planning their day."
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=120,
            num_return_sequences=1,
            temperature=0.9,
            top_p=0.95,
            do_sample=True
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Streamlit UI
st.title("AI-Powered Weather Forecast App")

city = st.text_input("Enter city name:")
if st.button("Get Weather"):
    weather_info = get_weather(city)
    if "Error" in weather_info:
        st.error(weather_info["Error"])
    else:
        classified_condition = classify_weather(weather_info["Weather"])
        summary = generate_weather_summary(weather_info)

        st.success(f"Weather in {weather_info['City']}:")
        st.write(f"🌡 Temperature: {weather_info['Temperature']} °C")
        st.write(f"💧 Humidity: {weather_info['Humidity']}%")
        st.write(f"🌦 Condition: {weather_info['Weather']} ({classified_condition})")

        st.write("📢 AI-Generated Weather Summary & Suggestions:")
        st.write(summary)


Writing app.py


In [25]:
!pip install pyngrok

In [26]:
!pip install streamlit pyngrok --quiet

In [27]:
!ngrok authtoken 2sv4epsyhLtljocv0phLU4sQJbk_377t99h7jrQKXYh4kuX6N

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [28]:
!curl https://loca.lt/mytunnelpassword

34.169.110.11

In [29]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧your url is: https://petite-pears-flow.loca.lt
